# Initialize stuff first

In [1]:
import scipy.integrate as sp1
import scipy.interpolate as sp2
import scipy.misc as sp3
from scipy import interpolate

import sys

import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import numba as nb
import os
from pathlib import Path
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

from matplotlib import rcParams
rcParams['figure.figsize'] = 8,8

sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

pardir=Path(os.getcwd()).parent.parent
datadir=str(pardir) +'/Data'

mycmap=sns.cubehelix_palette(n_colors=50, start=1, rot=4, light=0.7, dark=0)

In [2]:
# my homebrew 2d interpolation function
@nb.autojit(nopython=True)
def int2d(x,y,matrix):
    a,b=matrix.shape
    left=np.int(min(max(np.int(x),0),a-2))
    right=left+1
    bottom=np.int(min(max(np.int(y),0),b-2))
    top=bottom+1
    x=right-x
    y=top-y
    return x*y*matrix[left,bottom]+(1-x)*y*matrix[right,bottom]+x*(1-y)*matrix[left,top]+(1-x)*(1-y)*matrix[right,top]

# this copy to pass into the integrator (scipy won't take the raw function)
@nb.autojit()
def int2dslow(x,y,matrix):
    a,b=matrix.shape
    left=np.int(min(max(np.int(x),0),a-2))
    right=left+1
    bottom=np.int(min(max(np.int(y),0),b-2))
    top=bottom+1
    x=right-x
    y=top-y
    return x*y*matrix[left,bottom]+(1-x)*y*matrix[right,bottom]+x*(1-y)*matrix[left,top]+(1-x)*(1-y)*matrix[right,top]

In [3]:
@nb.autojit(nopython=True)
def create7Lin(reflect6,reflect7,fc,right,w,scale): 
    for i in range(0,100*fc+1):
        for j in range(i,min(i+right-100*fc,right)):
            bid=(1-w)*(j-i)/w
            if bid<100*fc+1 and bid>-1:
                reflect7[i,j]=int2d(i,bid,reflect6)

@nb.autojit(nopython=True)
def create7LogLin(reflect6,reflect7,fc,right,w,scale): 
    for i in range(0,100*fc+1):
        for j in range(i,right):
            iloc=i/(fc*scale)
            jloc=j/(fc*scale)
            aq=iloc-100
            th=jloc-100
            q=np.power(-(100)/aq,1/w)
            rmc=-100*(jloc-iloc)/aq
            jac=np.power(q,w+1)/(-aq*w)
            reflect7[i,j]=int2d(q*fc,rmc*fc,reflect6)*jac

In [4]:
@nb.autojit(nopython=True)
def create1(Fp1,right,fc): 
    for j in range(0,right):
        for i in range(0,100*fc):
            Fp1[i+1,j]+=Fp1[i,j]

@nb.autojit(nopython=True)
def create2(Fp2,right,fc): 
    for i in range(0,100*fc+1):
        for j in range(0,right-1):
            Fp2[i,j+1]+=Fp2[i,j]

@nb.autojit(nopython=True)
def create0(F,right,fc): 
    for i in range(0,100*fc+1):
        for j in range(0,right-1):
            F[i,j+1]+=F[i,j]

@nb.autojit(nopython=True)
def createZ(Z,F,right,freq,powr,fc): 
    FF=F/np.max(F)
    for i in range(0,100*fc+1):
        for j in range(0,right):
            num=0
            den=0
            for k in range(0,len(powr)):
                num+=np.power(FF[i,j],powr[k]-2)*freq[k]
                den+=np.power(FF[i,j],powr[k]-2)*freq[k]*(powr[k]-1)
            Z[i,j]=num/den

@nb.autojit(nopython=True)
def create3(Z,F,Fp1,Fp2,rhs,rhsfake,right,fc): 
    for i in range(0,100*fc+1):
        for j in range(i+1,right):
            den=-Fp2[i,j]+Z[i,j]*F[i,j]/(j-i)
            if den!=0:
                my=Fp1[i,j]/den
                rhs[i,j]=my
                if my>0:
                    rhsfake[i,j]=1
            rhs[0,0]=1/2
            rhs[0,1]=1/2
            
@nb.autojit(nopython=True)
def create8(reflect7,reflect8,myinv,myfor,F,xmax,ymax,fc,right): 
    for i in range(0,100*fc+1):
        myright=right#min(i+int(round(100*fc*ww)),right)
        for j in range(i,myright):
            if i<ymax:
                if j<xmax:
                    inv=myinv[i]
                    if inv>=j:
                        reflect8[i,j]=F[i,inv]
                    else:
                        reflect8[i,j]=F[myfor[j],j]
                else:
                    reflect8[i,j]=F[100*fc,j]
            else:
                reflect8[i,j]=F[i,right-1]
                
@nb.autojit(nopython=True)
def create9(reflect8,reflect9, l, powers, frequencies, right, fc): 
    for k in range(0,l):
        pk=powers[k]-1
        fk=frequencies[k]
        for i in range(0,100*fc+1):
            myright=right#i+int(round(100*fc*ww))
            for j in range(i,myright):
                reflect9[i,j]+=(reflect8[i,j]**pk)*fk
                
@nb.autojit(nopython=True)
def myhelpLin(rrr, qqq, fc, ww): 
    for i in range(0,100*fc+1):
        myright=i+int(round(100*fc*ww))
        for j in range(i,myright):
            qqq[i,j]=i/fc
            rrr[i,j]=1/(ww*fc)
            
@nb.autojit(nopython=True)
def myhelpLogLin(rrr, qqq, fc, scale,w,right): 
    for i in range(0,100*fc+1):
        iloc=i/(fc*scale)
        aq=iloc-100
        q=np.power(-(100)/aq,1/w)
        for j in range(0,right):
            rrr[i,j]=np.power(q,w)/(fc*scale)
            qqq[i,j]=q 
            
@nb.autojit(nopython=True)
def createR(reflect9,reflect9help,rebate,myinv,right,fc): 
    end=min(100*fc+1,len(myinv))
    for i in range(0,end):
        myright=right
        for j in range(i,myright-1):
            reflect9help[i,j+1]+=reflect9help[i,j]
        for j in range(myinv[i],myright):
            rebate[i,j]=max((j-i)*reflect9[i,j]-reflect9help[i,j],0)
            
@nb.autojit(nopython=True)
def mymultiply(reflect9,rebate,quality): 
    mysum=np.sum(reflect9)
    EQ=np.sum(np.multiply(reflect9,quality))/mysum
    ER=np.sum(np.multiply(reflect9,rebate))/mysum
    return ER,EQ

In [5]:
fc=8

for design in ['linear']:
    print('design=',design)
    for industry in ["leg", "mil", "sci", "edu", "tec"]:
        print('industry=',industry)
        for weight in ['55','80']:
            print('weight=',weight)
            mydir=datadir+"/derive/"+industry+weight+"/"
            dist_rmcq=pkl.load(open(mydir+industry+weight+"j_rcmq_"+str(fc)+".pkl", "rb"))
            
            try:
                [PART,DATA]=pkl.load(open(datadir+'/derive/'+industry+weight+"j.pkl", "rb"))
                powr=np.array(PART[0])
                freq=np.array(PART[1])
                freq=freq/np.sum(freq)
            except:
                powr=np.array([2])
                freq=np.array([1])
                          
            num=0
            l=len(powr)
            for i in range(0,l):
                num+=powr[i]*freq[i]

            curve={}
            for w in list(np.array(range(20,190))/50):
                w=w/4 

                #sys.stdout.write('7..., ')

                if design=='linear':
                    ww=w/(1-w)
                    scale=1/1-w
                    right=int(100*fc/(1-w))+1
                    dist_alth=np.zeros((100*fc+1,right),dtype=float)
                    create7Lin(dist_rmcq,dist_alth,fc,right,w,scale)
                elif design=='loglinear':
                    scale=1/(1-(1/np.power(100,w)))        
                    right=int((100*fc)*scale)+1
                    dist_alth=np.zeros((100*fc+1,right),dtype=float)
                    create7LogLin(dist_rmcq,dist_alth,fc,right,w,scale)

                Fp1=dist_alth.copy(); create1(Fp1,right,fc)
                Fp2=dist_alth.copy(); create2(Fp2,right,fc)
                F=Fp1.copy(); create0(F,right,fc)

                Z=np.zeros((100*fc+1,right),dtype=float) 
                createZ(Z,F,right,freq,powr,fc)

                rhsfake=np.zeros((100*fc+1,right),dtype=float)                            
                rhs=np.zeros((100*fc+1,right),dtype=float)
                create3(Z,F,Fp1,Fp2,rhs,rhsfake,right,fc)   

                mydic={}

                def myfun(x,y):
                    return int2dslow(y,x,rhs)

                myode=sp1.ode(myfun, jac=None)
                myode.set_initial_value(1*fc, 1*fc+1)
                myode.set_integrator('dopri5', nsteps=5000)

                dt=0.1
                x=[1*fc,1*fc]
                y=[1*fc+1,1*fc+1]

                flag=0

                #sys.stdout.write('solving ODE... ')

                while y[-1] < 100*fc+1 and x[-1] < right:
                    newy=myode.integrate(myode.t+dt)[0]
                    if newy-y[-1]>10*dt or (newy==y[-1] and y[-1]>2*fc):
                        x=x+[x[-1]]
                        y=y+[y[-1]+dt]
                        myode.set_initial_value(y[-1], x[-1])
                    else:
                        x=x+[myode.t+dt]
                        y=y+[newy]

                mydic[int(round(1000*w))]=[np.array(x),np.array(y)]

                fstrat = interpolate.interp1d([0]+list(x),[0]+list(y))
                fstrat_inv = interpolate.interp1d([0]+list(y),[0]+list(x))

                ymax=int(max(y))
                #print('ymax=',ymax)

                xmax=int(max(x))
                #print('xmax=',xmax)

                myinv=[]
                for i in range(0,ymax):
                    myinv+=[int(fstrat_inv(i))]
                myinv=np.array(myinv)

                myfor=[]
                for i in range(0,xmax):
                    myfor+=[int(fstrat(i))]
                myfor=np.array(myfor)

                dist_pseudo=np.zeros((100*fc+1,right),dtype=float)

                #sys.stdout.write('8..., ')

                create8(dist_alth,dist_pseudo,myinv,myfor,F,xmax,ymax,fc,right)
                dist_pseudo=dist_pseudo/np.max(dist_pseudo)   

                dist_power=np.zeros((100*fc+1,right),dtype=float)

                #sys.stdout.write('9..., ')

                create9(dist_pseudo,dist_power, l ,powr, freq, right, fc)

                rrr=np.zeros((100*fc+1,right),dtype=float)      
                qqq=np.zeros((100*fc+1,right),dtype=float)

                if design=='linear':
                    myhelpLin(rrr, qqq, fc, ww)
                if design=='loglinear':
                    myhelpLogLin(rrr, qqq, fc, scale,w,right)

                quality=np.multiply(dist_power,qqq)
                #print(np.min(quality),np.max(quality))

                rebate=np.zeros((100*fc+1,right),dtype=float)

                dist_power_help=dist_power.copy()

                createR(dist_power,dist_power_help,rebate,myinv,right,fc)
                rebate=np.multiply(rebate,rrr)

                ER,EQ=mymultiply(dist_alth,rebate,quality)
                sys.stdout.write('w='+str(w)+': ER='+str(round(num*ER,2))+', EQ='+str(round(num*EQ,2))+' || ')
                curve[int(round(1000*w))]=[num*ER,num*EQ]
            print('saving...')        
            addr=mydir+industry+weight+"j_curve_"+design+".pkl"
            pkl.dump(curve,open(addr, "wb"))

design= linear
industry= leg
weight= 55
w=0.1: ER=0.06, EQ=92.73 || w=0.105: ER=0.07, EQ=92.73 || w=0.11: ER=0.08, EQ=92.73 || w=0.115: ER=0.09, EQ=92.73 || w=0.12: ER=0.1, EQ=92.43 || w=0.125: ER=0.11, EQ=92.44 || w=0.13: ER=0.12, EQ=92.72 || w=0.135: ER=0.13, EQ=92.72 || w=0.14: ER=0.14, EQ=92.47 || w=0.145: ER=0.16, EQ=92.72 || w=0.15: ER=0.17, EQ=92.49 || w=0.155: ER=0.19, EQ=92.72 || w=0.16: ER=0.2, EQ=92.5 || w=0.165: ER=0.22, EQ=92.51 || w=0.17: ER=0.24, EQ=92.71 || w=0.175: ER=0.25, EQ=92.71 || w=0.18: ER=0.27, EQ=92.71 || w=0.185: ER=0.29, EQ=92.71 || w=0.19: ER=0.32, EQ=92.71 || w=0.195: ER=0.34, EQ=92.7 || w=0.2: ER=0.36, EQ=92.54 || w=0.205: ER=0.38, EQ=92.54 || w=0.21: ER=0.41, EQ=92.7 || w=0.215: ER=0.44, EQ=92.55 || w=0.22: ER=0.47, EQ=92.69 || w=0.225: ER=0.49, EQ=92.55 || w=0.23: ER=0.52, EQ=92.69 || w=0.235: ER=0.55, EQ=92.69 || w=0.24: ER=0.59, EQ=92.68 || w=0.245: ER=0.62, EQ=92.68 || w=0.25: ER=0.65, EQ=92.68 || w=0.255: ER=0.69, EQ=92.67 || w=0.26: ER=0.73, EQ=92.